In [2]:
import requests
from bs4 import BeautifulSoup
import re
import os
import pandas as pd
from lxml import etree
from datetime import datetime
import openpyxl
from openpyxl import Workbook
from openpyxl import load_workbook
current_directory = os.getcwd()

In [13]:

def get_ma_so_link_info(link):
    print(link)
    response2 = requests.get(link.strip())
    soup = BeautifulSoup(response2.text, 'html.parser')
    content1 = soup.find('div', class_='content1')
    tables = content1.find_all('table')

    # lấy số văn bản / ngày
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'Số:' in text:
                số = text.replace("Số:", "").strip()
                # print(số)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'Số'.")
    except AttributeError as e:
            print(f"Lỗi: {e}")
    try:
        paragraphs = tables[0].find_all('p')  # Tìm tất cả các thẻ <p> trong bảng đầu tiên
        for para in paragraphs:
            text = para.get_text(strip=True).replace('\r', '').replace('\n', '').strip()
            if 'ngày' in text:
                ngày = normalize_date(text)
                # print(ngày)
                break  # Ngừng vòng lặp sau khi tìm thấy đoạn văn bản chứa 'ngày'
        else:
            print("Không tìm thấy đoạn văn bản chứa 'ngày'.")
    except AttributeError as e:
        print(f"Lỗi: {e}")
    info_list = [link.strip(),số,ngày]

    # lấy part 1
    part1 = tables[1].find_all('p')
    part1_part_1  = next((i for i, item in enumerate(part1) if "1. Hàng hóa " in item.get_text(strip=True)), -1)
    part1_part_1_text  = part1[part1_part_1].get_text().replace('\r', '').replace('\n', '').strip()
    part1_part_2 = next((i for i, item in enumerate(part1) if "Tên thương mại" in item.get_text(strip=True)), -1)
    part1_part_2_text  = part1[part1_part_2].get_text().replace('\r', '').replace('\n', '').strip()
    part1_part_3 = next((i for i, item in enumerate(part1) if "Tên gọi theo cấu tạo" in item.get_text(strip=True)), -1)
    part1_part_3_text  = part1[part1_part_3].get_text().replace('\r', '').replace('\n', '').strip()
    part1_part_4 = next((i for i, item in enumerate(part1) if "mã hiệu" in item.get_text(strip=True)), -1)
    part1_part_4_text  = part1[part1_part_4].get_text().replace('\r', '').replace('\n', '').strip()
    part1_part_5 = next((i for i, item in enumerate(part1) if "Nhà sản xuất" in item.get_text(strip=True)), -1)
    part1_part_5_text  = part1[part1_part_5].get_text().replace('\r', '').replace('\n', '').strip()
    info_list.append(part1_part_2_text)
    info_list.append(part1_part_3_text)
    info_list.append(part1_part_4_text)
    info_list.append(part1_part_5_text)

    # allpart
    table_temp = tables[1]
    for table in tables[1].find_all('table'):
        # Kiểm tra phần tử liền trước của thẻ <table>
        prev_sibling = table.find_previous_sibling()
        if prev_sibling and prev_sibling.name == 'p':
            prev_sibling.decompose()  # Xóa thẻ <p> liền trước
        table.decompose()  # Xóa thẻ <table>
    allpart = table_temp.find_all('p')
    
    # print("allpart: ",allpart)

    
    # lấy part 3
    pattern = re.compile(r'\d{4}\s*\.\s*\d{2}\s*\.\s*\d{2}')
    for p_tag in allpart:
        text = p_tag.get_text(strip=True)  # Lấy văn bản từ thẻ <p> và loại bỏ khoảng trắng thừa
        text_2 = p_tag.get_text()
        matches = pattern.findall(text)  # Tìm tất cả các dãy số khớp với pattern trong văn bản
        if matches:
            part3 = matches[-1]
            part3_text = text_2.replace('\r', '').replace('\n', '').strip()
    # lấy part 2
    try :
        part2_start = next((i for i, item in enumerate(allpart) if re.search(r"2\s*\.\s*Tóm tắt", item.get_text())), -1)-1
        # print(part2_start)
    except:
        part2_start = next((i for i, item in enumerate(allpart) if "Nhà sản xuất" in item.get_text()), -1)
        # print(part2_start)
    part2_end = next((i for i, item in enumerate(allpart) if re.search(r"3\s*\.\s*Kết quả", item.get_text(strip=True))), -1)
    part2_list = []
    for i in allpart[part2_start+1:part2_end]:
        part2_list.append(i.get_text().replace('\r', '').replace('\n', '').strip())
    part2_list
    part2_text = '\n'.join(part2_list)

    info_list.append(part2_text)
    info_list.append(part3)
    info_list.append(part3_text)
    
    info_list = [re.sub(r'[ ]+', ' ', text).strip() for text in info_list]
    
    print(info_list)
    return info_list

In [4]:
def normalize_date(text):
    # Biểu thức chính quy để bắt ngày, tháng, năm
    pattern = r"(\d{1,2})\s*tháng\s*(\d{1,2})\s*năm\s*(\d{4})"
    match = re.search(pattern, text)
    
    if match:
        # Lấy ra các thành phần ngày, tháng, năm
        day, month, year = match.groups()
        
        # Chuyển đổi thành định dạng dd-mm-yyyy
        date_obj = datetime.strptime(f"{day}-{month}-{year}", "%d-%m-%Y")
        return date_obj.strftime("%d-%m-%Y")
    return None

In [5]:
with open('tesst_link.txt', 'r', encoding='utf-8') as file:
    all_links = file.readlines()

In [6]:
len(all_links)

736

In [12]:
all_maso_info_list = []
for link in all_links:
    # if 'truoc-ma' in link:
    if link:
        try:
            info_list = get_ma_so_link_info(link)
            all_maso_info_list.append(info_list)
            # write_data_to_excel(maso_sheet,info_list)
        except:
            print(link + "lỗi")
            info_list = [link,"lỗi"]
            all_maso_info_list.append([link,"lỗi"])
            # write_data_to_excel(phanloai_sheet,info_list)

https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3664-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-Stipsipeg-baby-619749.aspx

['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-3664-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-Stipsipeg-baby-619749.aspx', '3664/TB-TCHQ', '29-07-2024', 'Tên thương mại: Stipsipeg baby', 'Tên gọi theo cấu tạo, công dụng: Stipsipeg baby', 'Ký, mã hiệu, chủng loại: không có', 'Nhà sản xuất: S.I.I.T S.r.l', '2. Tóm tắt mô tả hàng hóa được  xác định trước mã số: Theo hồ sơ xác định trước mã số, thông  tin mặt hàng như sau:\n- Thành phần, cấu tạo, công thức hóa học:\n+ Macrogol 3350.\n+ Tá dược: sillicon dioxide, hương liệu, natri saccharin\n- Cơ chế hoạt động, cách thức sử dụng:\n+ Stipsipeg baby đặc biệt thích hợp trong trường  hợp táo bón cấp tính và mãn tính thông qua tác dụng nhuận tràng thẩm thấu,  làm mềm phân, tạo điều kiện thuận lợi cho việc đi tiêu một cách tự nhiên mà  không gây ra hiện tượng co rút, đau bụng hay nguy hiểm đến sức k

In [8]:
get_ma_so_link_info("https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-5628-TB-TCHQ-2023-xac-dinh-truoc-ma-so-hang-hoa-585862.aspx")

https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-5628-TB-TCHQ-2023-xac-dinh-truoc-ma-so-hang-hoa-585862.aspx
['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-5628-TB-TCHQ-2023-xac-dinh-truoc-ma-so-hang-hoa-585862.aspx', '5628/TB-TCHQ', '01-11-2023', 'Tên thương mại: Khung giường bằng thép QUEEN    SLEEPER MECHANISM LP.', 'Tên gọi theo cấu tạo, công dụng: Khung giường bằng    thép, có gắn các lò xo bằng thép kết nối với mặt giường bằng vải bố, được    thiết kế làm thành giường, có thể gấp lại hoặc mở ra và gắn vào ghế sofa    (có chức năng chuyển thành giường). Sau đây gọi chung là “Khung giường bằng    thép”', 'Ký, mã hiệu, chủng loại: MRP# 77937 (SLEEPER    MECHANISM LP)', 'Nhà sản xuất: Zhejiang Chumpen Smart Home Co.,    Ltd (China).', '2. Tóm tắt mô tả hàng hóa được xác định trước  mã số:\nTheo hồ sơ đề nghị xác định trước mã số, thông  tin mặt hàng như sau:\n- Thành phần, cấu tạo: Khung giường bằng thép, có  gắn các lò xo bằng thép kết nối với mặt giường bằ

['https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-5628-TB-TCHQ-2023-xac-dinh-truoc-ma-so-hang-hoa-585862.aspx',
 '5628/TB-TCHQ',
 '01-11-2023',
 'Tên thương mại: Khung giường bằng thép QUEEN SLEEPER MECHANISM LP.',
 'Tên gọi theo cấu tạo, công dụng: Khung giường bằng thép, có gắn các lò xo bằng thép kết nối với mặt giường bằng vải bố, được thiết kế làm thành giường, có thể gấp lại hoặc mở ra và gắn vào ghế sofa (có chức năng chuyển thành giường). Sau đây gọi chung là “Khung giường bằng thép”',
 'Ký, mã hiệu, chủng loại: MRP# 77937 (SLEEPER MECHANISM LP)',
 'Nhà sản xuất: Zhejiang Chumpen Smart Home Co., Ltd (China).',
 '2. Tóm tắt mô tả hàng hóa được xác định trước mã số:\nTheo hồ sơ đề nghị xác định trước mã số, thông tin mặt hàng như sau:\n- Thành phần, cấu tạo: Khung giường bằng thép, có gắn các lò xo bằng thép kết nối với mặt giường bằng vải bố, được thiết kế làm thành giường (có thể gấp lại hoặc mở ra. Được tách biệt, trình bày riêng biệt thành giường khi không gắn với

In [9]:
df = pd.DataFrame(all_maso_info_list)

In [10]:
df.to_excel("result.xlsx",index = False)